<h3> 현재 구축 상황 </h3>
폴더에서 파일 불러와서 result란 폴더를 생성 
음원 파일이 있는 폴더에 대해 mfcc 연산을 하고 그 plot을 result 폴더에 저장할 수 있다.<br>
mfcc 데이터 csv file로 저장 가능<br>
csv format의 data_description을 통해서 mfcc data를 emotion에 따라 여러 폴더에 나눠서 저장<br> 

<h3> 해야할 것 </h3>

병렬 연산에 되면 무조건 찾아봐야 함. 데이터 너무 많음.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import matplotlib.cm as cm
import pandas as pd

In [2]:
path = 'C:/Users/RestJSM/librosa/data/emotion_dataset/5th_year/data'

audio_list = os.listdir(path)

In [16]:
pd_path = 'C:/Users/RestJSM/librosa/data/emotion_dataset'
pd_csv = pd.read_csv(pd_path + '/5th_year.csv', encoding = 'cp949')
pd_csv_id = pd_csv.loc[:, ['wav_id', '상황']]
print(pd_csv_id)

                         wav_id       상황
0      5ed10dbc2880d70f286121c3  disgust
1      5ecb60ef9aa8ea0eec53edb1  disgust
2      5f052858b140144dfcfef768      sad
3      5f0e72c4b140144dfcff3fa5      sad
4      5ed10ddd7e21a10eee2537ce  disgust
...                         ...      ...
10006  5f05ffd5b140144dfcff01c7     fear
10007  5ec53bf82880d70f28611de0      sad
10008  5f0b2b5ab140144dfcff2759     fear
10009  5f0b2b75b140144dfcff275a     fear
10010  5f0527b3b140144dfcfef75e     fear

[10011 rows x 2 columns]


In [4]:
# 문자로 데이터 찾기
def find_emotion(csvfile, str): 
    return csvfile[csvfile['wav_id'].str.contains(str)].iloc[0][1]


In [17]:
# 폴더 생성
## 출처 : https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


createFolder(path + '/res_anger')
createFolder(path + '/res_sad')
createFolder(path + '/res_neutral')
createFolder(path + '/res_disg')
createFolder(path + '/res_fear')


In [21]:
cnt=1
for audio in audio_list:
    # 경로 확장자 분리 참조 : https://jvvp.tistory.com/980
    audio_path = path + '/' + audio_list[cnt-1]
    file_code, ext = os.path.splitext(audio_path) # 확장자 분리
    dirname, basename = os.path.split(file_code) # 경로 분리
    y, sr = librosa.load(audio_path, 16000)
    
    
    frame_length = 0.025
    frame_stride = 0.01
    sr = 16000

    n_fft_25 = int(round(sr * frame_length))
    hop_length_10 = int(round(sr * frame_stride))
    
    n_fft = n_fft_25 * 2
    win_length = n_fft_25
    hop_length = hop_length_10
    n_mels = 128 # default
    n_mfcc = 20 # default

    # mfcc에는 멜 스펙트로그램의 log값이 필요하다. 
    ## 원래는 logamplitude 함수로 변환하였으나, power를 db로 변환하는 과정 자체가 로그함수를 씌우는 것이기 때문에 power_to_db로 함수가 대체됨
    
    # audio file -> MFCC data
    D = np.abs(librosa.stft(y, n_fft=n_fft, win_length = win_length, hop_length=hop_length))
    MS = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(D), sr=sr, n_mfcc=n_mfcc)
    
    
    # numpy array csv로 추출 참조 : https://appia.tistory.com/204
    if find_emotion(pd_csv_id, basename) == 'anger':
        ext_name = path + '/res_anger/'+basename+'.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    elif find_emotion(pd_csv_id, basename) == 'sad':
        ext_name = path + '/res_sad/'+basename+'.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    elif find_emotion(pd_csv_id, basename) == 'disgust':
        ext_name = path + '/res_disg/'+basename+'.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    elif find_emotion(pd_csv_id, basename) == 'fear':
        ext_name = path + '/res_fear/'+basename+'.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    else :
        ext_name = path+'/result/'+basename+'.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    
    
    # to plot(visualization)
#     plt.figure(figsize=(10, 4))
#     fig, ax = plt.subplots()
#     img = librosa.display.specshow(mfccs, x_axis='time', ax=ax)
#     fig.colorbar(img, ax=ax)
#     ax.set(title='MFCC')
#     plt.savefig(path + '/result/' + basename)
#     plt.close(fig)
    
    
    print(str(cnt) + "/" + str(len(audio_list) - 1))
    cnt = cnt + 1
    if cnt == len(audio_list):
        print("MFCC transformation is finished!")
    

1/10010
2/10010
3/10010
4/10010
5/10010
6/10010
7/10010
8/10010
9/10010
10/10010
11/10010
12/10010
13/10010
14/10010
15/10010
16/10010
17/10010
18/10010
19/10010
20/10010
21/10010
22/10010
23/10010
24/10010
25/10010
26/10010
27/10010
28/10010
29/10010
30/10010
31/10010
32/10010
33/10010
34/10010
35/10010
36/10010
37/10010
38/10010
39/10010
40/10010
41/10010
42/10010
43/10010
44/10010
45/10010
46/10010
47/10010
48/10010
49/10010
50/10010
51/10010
52/10010
53/10010
54/10010
55/10010
56/10010
57/10010
58/10010
59/10010
60/10010
61/10010
62/10010
63/10010
64/10010
65/10010
66/10010
67/10010
68/10010
69/10010
70/10010
71/10010
72/10010
73/10010
74/10010
75/10010
76/10010
77/10010
78/10010
79/10010
80/10010
81/10010
82/10010
83/10010
84/10010
85/10010
86/10010
87/10010
88/10010
89/10010
90/10010
91/10010
92/10010
93/10010
94/10010
95/10010
96/10010
97/10010
98/10010
99/10010
100/10010
101/10010
102/10010
103/10010
104/10010
105/10010
106/10010
107/10010
108/10010
109/10010
110/10010
111/1001

KeyboardInterrupt: 